This is a tutorial pyflowline notebook.
This tutorial is an example of the pyflowline application using a Model for Prediction Across Scales (MPAS) mesh.

The following publication includes a comprehensive application:
Liao, C., Zhou, T., Xu, D., Cooper, M. G., Engwirda, D., Li, H.-Y., & Leung, L. R. (2023). Topological relationship-based flow direction modeling: Mesh-independent river networks representation. Journal of Advances in Modeling Earth Systems, 15, e2022MS003089. https://doi.org/10.1029/2022MS003089

The full documentation is hosted at: https://pyflowline.readthedocs.io

In order the run this notebook, you must install the PyFlowline package and its dependencies. 
Besides, the visulization requires the optional dependency packages (see the full documentation installation section).
You can also modify the notebook to use a different visualization method.


First, let's load some Python libraries.

In [ ]:
#step 1
import os
import sys
from pathlib import Path
from os.path import realpath
import importlib
#install dependencies

iFlag_numpy = importlib.util.find_spec("numpy")
iFlag_gdal = importlib.util.find_spec("osgeo")
iFlag_netcdf4 = importlib.util.find_spec("netCDF4")
iFlag_geopandas = importlib.util.find_spec("geopandas")
iFlag_requests = importlib.util.find_spec("requests") #this one is only for downloading the data

In [ ]:
if iFlag_numpy is not None:
    pass
else:
    print("numpy is not installed")

In [ ]:
if iFlag_gdal is not None:
    pass
else:
    print("gdal is not installed")

In [ ]:
if iFlag_netcdf4 is not None:
    pass
else:
    print("netCDF4 is not installed")

In [ ]:
if iFlag_geopandas is not None:
    pass
else:
    print("geopandas is not installed")

In [ ]:
if iFlag_requests is not None:
    pass
else:
    print("requests is not installed")

import requests

Then we will check whether some additional packages are installed.

In [ ]:
sPath_notebook = Path().resolve()
sPath_parent = str(Path().resolve().parents[0]) 
print(sPath_parent)
#check pyflowline installation
iFlag_pyflowline = importlib.util.find_spec("pyflowline") 
if iFlag_pyflowline is not None:
    pass
else:
    print('The pyflowline package is not installed. We will use the current path to set it up.')
    sys.path.append(sPath_parent)




We need to download an additional NetCDF file for this example.
This file is stored on the Github release:
https://github.com/changliao1025/pyflowline/releases/tag/0.2.0
https://github.com/changliao1025/pyflowline/releases/download/0.2.0/lnd_cull_mesh.nc

In [ ]:

#download the MPAS mesh from the github release
sFilename_mpas = 'https://github.com/changliao1025/pyflowline/releases/download/0.2.0/lnd_cull_mesh.nc'

#combind folder with filename to get the full path
sFolder_data = os.path.join(sPath_parent, 'data')
sFolder_data_susquehanna =  os.path.join(sFolder_data, 'susquehanna')
sFolder_input = os.path.join(sFolder_data_susquehanna, 'input')

sFilename_download = os.path.join(sFolder_input, 'mpas_mesh.nc')
print(sFilename_download)

# Send an HTTP GET request to the URL
response = requests.get(sFilename_mpas)

# Check if the request was successful
if response.status_code == 200:
    # Save the content of the response to the local file
    with open(sFilename_download, 'wb') as file:
        file.write(response.content)
    print(f"File '{sFilename_download}' downloaded successfully.")
else:
    print(f"Failed to download file from '{sFilename_mpas}'.")

Now we can import a few python function within pyflowline.
The pyflowline python package uses the OOP approach to manage each simulation.

In [ ]:
#step 3
#load the read configuration function
from pyflowline.change_json_key_value import change_json_key_value
from pyflowline.pyflowline_read_model_configuration_file import pyflowline_read_model_configuration_file

pyflowline uses a json file for configuration, an example json file is provided.
check whether a configuration exists

In [ ]:
sFilename_configuration_in = realpath( sPath_parent +  '/examples/susquehanna/pyflowline_susquehanna_mpas.json' )
if os.path.isfile(sFilename_configuration_in):
    pass
else:
    print('This configuration does not exist: ', sFilename_configuration_in )

In [ ]:
#step 4B(optional)
#we can check what is the content of this json file
import json
with open(sFilename_configuration_in, 'r') as pJSON:
    parsed = json.load(pJSON)
    print(json.dumps(parsed, indent=4))

The meaning of these json keywords are explained in the pyflowline documentation: https://pyflowline.readthedocs.io/en/latest/data/data.html#inputs

For some parameters, we can change them using the following function call.

For some other parameters (e.g., path to file), you need to modify the json file using a text editor.

If the function returns an error, you should update the json file(s). 

Now set up some keywords

In [ ]:
#step 5, set up some parameters
sMesh_type = 'mpas'
iCase_index = 1
dResolution_meter=5000
sDate='20230101'

In [ ]:
#we need to update a few parameters in the configuration file before we can create the flowline object
sFilename_basins =  realpath( os.path.join(sFolder_input , 'pyflowline_susquehanna_mpas_basins.json' ))
sFilename_mesh_boundary = realpath(os.path.join(sFolder_input, 'boundary_wgs.geojson'))

change_json_key_value(sFilename_configuration_in, 'sFilename_mesh_netcdf', sFilename_download)
change_json_key_value(sFilename_configuration_in, 'sFilename_mesh_boundary', sFilename_mesh_boundary)
change_json_key_value(sFilename_configuration_in, 'sFilename_basins', sFilename_basins)


we can now call the function to create an object

In [ ]:
#the read function accepts several keyword arguments that can be used to change the default parameters.
#the normal keyword arguments are:
#iCase_index_in: this is an ID to identify the simulation case
#sMesh_type_in: this specifies the mesh type ('mpas' in this example)
#sDate_in: this specifies the date of the simulation, the final output folder will have a pattern such as 'pyflowline20230901001', where pyflowline is model, 20230901 is the date, and 001 is the case index.
oPyflowline = pyflowline_read_model_configuration_file(sFilename_configuration_in, iCase_index_in=iCase_index, 
                                                    sMesh_type_in= sMesh_type, sDate_in=sDate)

In [ ]:
#other than setting the paraemeter using the read model configuration fucntion, user can also change model parameters after creating the model object
#rememere that, it is recommended to set output folder using the read model configuration function since the change_model_parameter function will not update output folder
#only a list of parameters can be changed, for full list, please check the documentation
#in this example, we will change the mesh file name
#the function will check the data type, if incorrect data type is provided, it will raise an error
oPyflowline.change_model_parameter('sFilename_mesh_netcdf', sFilename_download) #because the mpas mesh already contains elevation, we do not need to set the elevation file name
#we will set the boundary file name
#this file should be located in the input folder
oPyflowline.change_model_parameter('sFilename_mesh_boundary', sFilename_mesh_boundary)

#we can also set for individual basin in the domain, in this example, we only has one basin.
#remember that, each basin can have different parameters, so if you want to set them different (for example, basin 1 has no dam, but basin 2 has dam), you should edit the basin json instead using this function.
#because change_model_parameter will set all the basin using the same parameter in current version
#must set iFlag_basin_in = 1 for basin parameter
oPyflowline.change_model_parameter('iFlag_dam', 0, iFlag_basin_in= 1)

#we will set the original flowline file name
sFilename_flowline = os.path.join(sFolder_input, 'flowline.geojson')
oPyflowline.change_model_parameter('sFilename_flowline_filtered', sFilename_flowline, iFlag_basin_in= 1)



You can review the setting again.

In [ ]:
print(oPyflowline.tojson())

If you are not certain of the outlet location, you can also set them up using:

In [ ]:
#another important setting for basin is the approximate outlet location
oPyflowline.aBasin[0].dLatitude_outlet_degree=39.4620
oPyflowline.aBasin[0].dLongitude_outlet_degree=-76.0093
#you can also set it using the change_model_parameter function
oPyflowline.change_model_parameter('dLongitude_outlet_degree', -76.0093, iFlag_basin_in = 1)
oPyflowline.change_model_parameter('dLatitude_outlet_degree', 39.4620, iFlag_basin_in = 1)

You can check the setting for the single basin as well

In [ ]:
print(oPyflowline.aBasin[0].tojson())

After the case object was created, we can set up the model.

In [ ]:
#setup the model       
oPyflowline.setup()

Before any operation, we can visualize the original or raw flowline dataset. 

You can either use your own visualization method to view or map the model output.

For example, if you want to use the built-in function, all you need to provide is a keyword (a look up table is provided in the documentation).
Be default, all the dataset will be converted to the WGS84 system to maintain consistency.
We can take a look at the user-provided flowline.

In [ ]:


if iFlag_geopandas == 1:
    import geopandas as gpd
    import matplotlib.pyplot as plt
    #use the geopanda package
    #the raw/original geojson file 
    sFilename_geojson = oPyflowline.aBasin[0].sFilename_flowline_filter_geojson
    gdf = gpd.read_file(sFilename_geojson)
    gdf.plot()
    plt.show()
else:
    print('The visulization packages are not installed.')
pass
    

You can also use QGIS or other Python libraries such as GeoPandas.

In [ ]:
#the raw/original geojson file 
sFilename_geojson = oPyflowline.aBasin[0].sFilename_flowline_filter_geojson


The plot function provides a few optional arguments such as map projection and spatial extent. 
By default, the spatial extent is full. 
But you can set the extent to a zoom-in region.

now let's run the three major steps/operations one by one.

In [ ]:
#run step 1
oPyflowline.flowline_simplification();

and check the result using a plot

In [ ]:
sFilename_geojson = oPyflowline.aBasin[0].sFilename_flowline_simplified_geojson
gdf = gpd.read_file(sFilename_geojson)
gdf.plot()
plt.show()
pass

Similarly, we can zoom in using the extent.

Next, we will creata a mesh from the global MPAS mesh.

In [ ]:
#run step 2
#we can either use a rectangle boundary
oPyflowline.iFlag_mesh_boundary = 0 #set to 0 to disable polygon-based
oPyflowline.dLongitude_left= -79
oPyflowline.dLongitude_right= -74.5
oPyflowline.dLatitude_bot= 39.20
oPyflowline.dLatitude_top= 42.8

aCell = oPyflowline.mesh_generation()

In [ ]:

#use the geopanda package
#the raw/original geojson file 
sFilename_geojson = oPyflowline.aBasin[0].sFilename_mesh_geojson
gdf = gpd.read_file(sFilename_geojson)
gdf.plot()
plt.show()
pass

we can also use a polygon to create a mesh

In [ ]:
oPyflowline.iFlag_mesh_boundary = 1
aCell = oPyflowline.mesh_generation()

In [ ]:
sFilename_geojson = oPyflowline.aBasin[0].sFilename_mesh_geojson
gdf = gpd.read_file(sFilename_geojson)
gdf.plot()
plt.show()

Last, we can generate the conceptual flowline.

In [ ]:
#runs step 3
oPyflowline.reconstruct_topological_relationship(aCell);

Now we can overlap mesh with flowline.

In [ ]:


    #
file1_path = oPyflowline.aBasin[0].sFilename_mesh_geojson
file2_path = oPyflowline.aBasin[0].sFilename_flowline_conceptual_geojson
gdf1 = gpd.read_file(file1_path)
gdf2 = gpd.read_file(file2_path)
fig, ax = plt.subplots()
gdf1.plot(ax=ax, color='blue')
gdf2.plot(ax=ax, color='red')
plt.show()
pass

and take a close look at the outlet region

After this, we can save the model output into a json file.

In [ ]:
#export output
oPyflowline.export();

the content of the one of the exported json files can be checked:

In [ ]:
with open(oPyflowline.sFilename_mesh_info, 'r') as pJSON:
    parsed = json.load(pJSON)
    print(json.dumps(parsed[0], indent=4))
    

In [ ]:
sFilename_flowline_conceptual_info= os.path.join(str(Path(oPyflowline.aBasin[0].sWorkspace_output_basin)  ), oPyflowline.aBasin[0].sFilename_flowline_conceptual_info  )    
                
with open(sFilename_flowline_conceptual_info, 'r') as pJSON:
    parsed = json.load(pJSON)
    print(json.dumps(parsed[0], indent=4))

The outlet associated flowline is always assigned with a dam, because it would be preserved.

Congratulations! You have successfully finished a pyflowline simulation.